## Image Similarity Scoring
Upload an image below or run as-is to use a sample image.

In [ ]:
import sys, subprocess
def _pip(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)
try:
    import cv2, skimage, PIL, numpy, pandas, matplotlib
except Exception as e:
    _pip(["opencv-python","scikit-image","Pillow","numpy","pandas","matplotlib"])

In [ ]:
import numpy as np, pandas as pd, cv2, io, math
from PIL import Image
from skimage import data, img_as_ubyte
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt

In [ ]:
try:
    from google.colab import files
    u = files.upload()
    n = next(iter(u.keys())) if len(u) else None
except Exception:
    n = None
if n:
    im = Image.open(io.BytesIO(u[n])).convert("RGB")
else:
    im = Image.fromarray(img_as_ubyte(data.astronaut()))
orig = np.array(im)

In [ ]:
def gray(x):
    return cv2.cvtColor(x, cv2.COLOR_RGB2GRAY) if x.ndim==3 else x
def ahash(x):
    g = cv2.resize(gray(x), (8,8), interpolation=cv2.INTER_AREA).astype(np.float32)
    m = g.mean()
    return (g>m).astype(np.uint8).flatten()
def dhash(x):
    g = cv2.resize(gray(x), (9,8), interpolation=cv2.INTER_AREA).astype(np.float32)
    d = g[:,1:] > g[:,:-1]
    return d.astype(np.uint8).flatten()
def phash(x):
    g = cv2.resize(gray(x), (32,32), interpolation=cv2.INTER_AREA).astype(np.float32)
    d = cv2.dct(g)
    d = d[:8,:8]
    med = np.median(d[1:].flatten())
    b = (d>med).astype(np.uint8)
    b[0,0] = 1
    return b.flatten()
def hsim(a,b):
    m = min(a.size,b.size)
    return 1.0 - (np.bitwise_xor(a.flatten()[:m], b.flatten()[:m]).sum()/m)
def ssim_sim(a,b):
    a = gray(a); b = gray(b)
    h = min(a.shape[0], b.shape[0]); w = min(a.shape[1], b.shape[1])
    return float(ssim(a[:h,:w], b[:h,:w]))
def orb_sim(a,b):
    a = gray(a); b = gray(b)
    orb = cv2.ORB_create(1000)
    k1,d1 = orb.detectAndCompute(a,None)
    k2,d2 = orb.detectAndCompute(b,None)
    if d1 is None or d2 is None or len(k1)==0 or len(k2)==0:
        return 0.0
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    m = bf.knnMatch(d1,d2,k=2)
    good = [p for p,q in m if len([p,q])==2 and p.distance<0.75*q.distance]
    return len(good)/max(len(k1),len(k2))
def hist_sim(a,b):
    ha = cv2.calcHist([cv2.cvtColor(a, cv2.COLOR_RGB2HSV)], [0,1,2], None, [8,8,8], [0,180,0,256,0,256])
    hb = cv2.calcHist([cv2.cvtColor(b, cv2.COLOR_RGB2HSV)], [0,1,2], None, [8,8,8], [0,180,0,256,0,256])
    ha = cv2.normalize(ha, ha).flatten()
    hb = cv2.normalize(hb, hb).flatten()
    v = cv2.compareHist(ha, hb, cv2.HISTCMP_CORREL)
    if np.isnan(v): 
        return 0.0
    return float((v+1)/2)


In [ ]:
def rotate(x,d):
    h,w = x.shape[:2]
    M = cv2.getRotationMatrix2D((w/2,h/2), d, 1.0)
    return cv2.warpAffine(x, M, (w,h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
def scale(x,fx,fy):
    return cv2.resize(x, (0,0), fx=fx, fy=fy, interpolation=cv2.INTER_AREA)
def gray3(x):
    return cv2.cvtColor(gray(x), cv2.COLOR_GRAY2RGB)
def blur(x,k):
    k = k+(k%2==0)
    return cv2.GaussianBlur(x,(k,k),0)
def sharpen(x):
    k = np.array([[0,-1,0],[-1,5,-1],[0,-1,0]], dtype=np.float32)
    return cv2.filter2D(x,-1,k)
def noise(x,s):
    r = np.random.randn(*x.shape).astype(np.float32)
    return np.clip(x.astype(np.float32)+s*r,0,255).astype(np.uint8)
def jpeg_q(x,q):
    buf = io.BytesIO()
    Image.fromarray(x).save(buf, format="JPEG", quality=q, optimize=True)
    buf.seek(0)
    return np.array(Image.open(buf).convert("RGB"))
t = {}
t["rotate_1"]=rotate(orig,1)
t["rotate_10"]=rotate(orig,10)
t["rotate_17"]=rotate(orig,17)
t["rotate_45"]=rotate(orig,45)
t["scale_0.5x"]=scale(orig,0.5,0.5)
t["scale_2x"]=scale(orig,2.0,2.0)
t["grayscale"]=gray3(orig)
t["blur_11"]=blur(orig,11)
t["sharpen"]=sharpen(orig)
t["noise_10"]=noise(orig,10)
t["jpeg_q30"]=jpeg_q(orig,30)

In [ ]:
def table(o, variants):
    rows=[]
    a0=ahash(o); d0=dhash(o); p0=phash(o)
    for k,v in variants.items():
        rows.append({"variant":k,"aHash":hsim(a0,ahash(v)),"dHash":hsim(d0,dhash(v)),"pHash":hsim(p0,phash(v)),"SSIM":ssim_sim(o,v),"ORB":orb_sim(o,v),"Histogram":hist_sim(o,v)})
    return pd.DataFrame(rows).sort_values("variant").reset_index(drop=True)
df = table(orig,t)
df

In [ ]:
for c in ["aHash","dHash","pHash","SSIM","ORB","Histogram"]:
    plt.figure(figsize=(8,4))
    plt.title(c)
    plt.ylim(0,1)
    plt.xticks(rotation=60, ha="right")
    plt.bar(df["variant"], df[c])
    plt.show()

In [ ]:
angles=[0,1,10,17,30,45,90]
rot = {f"rot_{a}":rotate(orig,a) for a in angles}
rdf = table(orig, rot)
metrics=["dHash","pHash","SSIM","ORB"]
plt.figure(figsize=(7,4))
for m in metrics:
    plt.plot(angles, [rdf.loc[rdf['variant']==f'rot_{a}', m].values[0] for a in angles], marker="o", label=m)
plt.xlabel("degrees")
plt.ylabel("similarity")
plt.legend()
plt.grid(True)
plt.show()
rdf

In [ ]:
cols=3
rows=(len(t)+cols-1)//cols
plt.figure(figsize=(15,5*rows))
i=1
for k,v in t.items():
    plt.subplot(rows,cols,i)
    plt.imshow(v)
    plt.title(k)
    plt.axis("off")
    i+=1
plt.tight_layout()
plt.show()

In [ ]:
for k,v in t.items():
    v_resized = cv2.resize(v, (orig.shape[1], orig.shape[0]), interpolation=cv2.INTER_AREA)
    d = gray(orig).astype("float32") - gray(v_resized).astype("float32")
    d = (np.abs(d) / np.max(np.abs(d))) * 255
    d = d.astype("uint8")
    plt.figure(figsize=(10,4))
    plt.subplot(1,3,1)
    plt.imshow(orig)
    plt.title("Original")
    plt.axis("off")
    plt.subplot(1,3,2)
    plt.imshow(v)
    plt.title(k)
    plt.axis("off")
    plt.subplot(1,3,3)
    plt.imshow(d, cmap="hot")
    plt.title("Difference heatmap")
    plt.axis("off")
    plt.tight_layout()
    plt.show()
